In [1]:
cd ..

/Users/fangxia/Library/Mobile Documents/com~apple~CloudDocs/code/hans_radish


In [2]:
import pandas as pd
from datetime import datetime, timedelta

In [3]:
from paprika.data.data_channel import DataChannel
from paprika.data.data_type import DataType

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/arctic/store/_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel


In [4]:
data = DataChannel.fetch_mp(['SP500..*'], data_type=DataType.CANDLE, frequency='1D')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pymongo/topology.py:150: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pymongo/topology.py:150: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pymongo/topology.py:150: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-

In [5]:
data.head()

open       high        low      close     volume  \
symbol  date                                                                
SP500.A 2007-01-03  25.028612  25.379112  24.356224  24.535049  3599200.0   
        2007-01-04  24.535049  24.749641  23.934191  24.613733  2899000.0   
        2007-01-05  24.535049  24.606581  24.320457  24.384836  3741800.0   
        2007-01-08  24.306152  24.377682  24.091558  24.298998  2176900.0   
        2007-01-09  24.377682  24.549356  24.055794  24.327610  1937900.0   

                   symbol  
symbol  date               
SP500.A 2007-01-03      A  
        2007-01-04      A  
        2007-01-05      A  
        2007-01-08      A  
        2007-01-09      A

In [6]:
from paprika.data.data_processor import DataProcessor
from paprika.data.feed_filter import TimeFreqFilter, TimePeriod

In [7]:
time_filter = TimeFreqFilter(TimePeriod.DAY, 1)
dp = DataProcessor(data)
dp.ohlcv(time_filter, inplace=True)

In [8]:
from paprika.alpha.base import Alpha
from paprika.alpha.utils import *

In [9]:
alpha = Alpha(dp)

In [10]:
def alpha_gt_1(dp: DataProcessor, period=6):
    '''(-1 * CORR(RANK(DELTA(LOG(VOLUME), 1)), RANK(((CLOSE - OPEN) / OPEN)), 6)) '''
    # df = input_df.copy()
    # df['volume'] = df['volume'].apply(np.log)
    # df['d_volume'] = df.groupby('ticker')['volume'].diff()
    # df['rank1'] = (df['d_volume'] * df['valid']).dropna().groupby(df['date']).rank(pct=True)
    # df['tmp'] = (df['close']-df['open']) /df['open']
    # df['rank2'] = (df['tmp'] * df['valid']).dropna().groupby(df['date']).rank(pct=True)
    # df['GT_1'] = qk_corr(df,'rank1','rank2',6) * -1
    # df['GT_1'] *= df['valid']
    # return df[['ticker','date','GT_1']]
    # -1 * CORR(RANK(DELTA(LOG(VOLUME), 1)), RANK(((CLOSE - OPEN) / OPEN)), 6)
    r1 = rank(delta(dp.volume.apply(np.log), 1))
    r2 = rank((dp.close - dp.open) / dp.open)
    return - correlation(r1, r2, period)

In [11]:
alpha.add_alpha(alpha_gt_1)

In [12]:
alpha['alpha_gt_1']

symbol,SP500.A,SP500.AAL,SP500.AAP,SP500.AAPL,SP500.ABBV,SP500.ABC,SP500.ABMD,SP500.ABT,SP500.ACN,SP500.ADBE,...,SP500.XEL,SP500.XLNX,SP500.XOM,SP500.XRAY,SP500.XRX,SP500.XYL,SP500.YUM,SP500.ZBH,SP500.ZION,SP500.ZTS
date,,,,,,,,,,,,,,,,,,,,,
2007-01-11,0.239837,0.240924,-0.476480,-0.539530,NaN,-0.162122,0.466153,-0.571971,-0.048615,0.557268,...,0.242855,-0.388094,0.790894,-0.621794,-0.596196,NaN,-0.354842,0.410312,0.295565,NaN
2007-01-12,0.172709,0.242085,-0.128195,-0.676752,NaN,-0.124291,0.212144,-0.331450,-0.371805,0.481537,...,0.257290,-0.400990,0.774547,-0.847732,-0.610537,NaN,-0.154310,0.658036,-0.353455,NaN
2007-01-16,0.055104,0.253057,-0.126353,-0.600728,NaN,-0.234507,0.161948,-0.728589,-0.689248,0.466441,...,0.030661,-0.263283,0.640198,-0.850791,-0.449656,NaN,-0.107738,0.505761,-0.184345,NaN
2007-01-17,-0.288936,0.557073,0.093123,-0.255256,NaN,-0.494930,0.421799,-0.939956,-0.423361,0.793523,...,-0.154206,-0.346741,0.074931,-0.623437,-0.339448,NaN,-0.114739,0.174816,-0.326466,NaN
2007-01-18,-0.441088,0.613088,0.133969,0.328123,NaN,-0.832601,0.057915,-0.969472,-0.334781,0.940019,...,-0.164964,-0.403186,-0.014735,-0.399507,-0.321369,NaN,0.160519,-0.055422,0.031483,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-05,0.466302,-0.798579,0.640785,-0.103082,0.875063,0.218514,0.810172,-0.548997,0.056256,-0.864731,...,-0.643311,-0.093679,-0.756757,-0.104266,-0.527442,-0.630022,-0.527081,-0.618899,0.104486,0.611824
2019-02-06,0.436276,-0.700131,0.737689,-0.682798,0.605591,0.211910,0.757342,-0.569078,-0.057976,-0.932435,...,-0.852728,-0.233975,-0.705102,-0.133232,-0.107045,-0.512920,-0.510918,-0.666110,0.644703,0.532288
2019-02-07,0.409601,-0.651122,0.534357,-0.565453,0.379174,-0.057920,0.902213,-0.360376,-0.010420,-0.645127,...,-0.844520,-0.216116,-0.648877,-0.162228,-0.432251,-0.296184,-0.603397,-0.615171,0.727526,0.139216


In [13]:
gspc = DataChannel.fetch(['GSPC.*'])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  """Entry point for launching an IPython kernel.


In [14]:
from paprika.alpha.beta import Beta

In [15]:
gspc.info

<bound method DataFrame.info of                           open         high          low        close  \
symbol date                                                             
GSPC   2001-01-22  1342.540039  1353.619995  1333.839966  1342.900024   
       2001-01-23  1342.900024  1362.900024  1339.630005  1360.400024   
       2001-01-24  1360.400024  1369.750000  1357.280029  1364.300049   
       2001-01-25  1364.300049  1367.349976  1354.630005  1357.510010   
       2001-01-26  1357.510010  1357.510010  1342.750000  1354.949951   
...                        ...          ...          ...          ...   
       2020-01-13  3271.129883  3288.129883  3268.429932  3288.129883   
       2020-01-14  3285.350098  3294.250000  3277.189941  3283.149902   
       2020-01-15  3282.270020  3298.659912  3280.689941  3289.290039   
       2020-01-16  3302.969971  3317.110107  3302.820068  3316.810059   
       2020-01-17  3323.659912  3329.879883  3318.860107  3329.620117   

                  

In [18]:
class Beta(object):
    def __init__(self):
        raise NotImplementedError
    def __getitem__(self, name: str) -> pd.DataFrame:
        return self._alpha[name].unstack('symbol')
    def list_beta(self) -> List[str]:
        return self._alpha.columns.to_list()

    def add_beta(self, df: pd.DattaFrame):
        name = alpha.__name__
        self._alpha_universe[name] = alpha
        if self._alpha is None:
            self._alpha = pd.DataFrame(alpha(self._dp, *args, **kwargs).stack('Symbol'),
                                       columns=[name])
        else:
            self._alpha[name] = alpha(self._dp, *args, **kwargs).stack('Symbol')


In [11]:
def alpha_101(dp, eps=0.001):
    return (dp.close - dp.open) / ((dp.high - dp.low) + eps)

def alpha_99(dp):
    cor1 = correlation(ts_sum((dp.high + dp.low) / 2, 19.8975), ts_sum(dp.adv(60), 19.8975), 8.8136)
    rank1 = rank(cor1)
    cor2 = correlation(dp.low, dp.volume, 6.28259)
    rank2 = rank(cor2)
    return - rank1 < rank2

def alpha_98(dp):
    cor1 = correlation(dp.vwap, ts_sum(dp.adv(5), 26.4719), 4.584818)
    decay1 = decay_linear(cor1, 7.18088)
    rank1 = rank(decay1)
    

In [12]:
alpha.add_alpha(alpha_101)

In [13]:
alpha.add_alpha(alpha_99)

In [14]:
alpha.corr_between_alpha_and_return()

,SP500.A,SP500.AAL,SP500.AAP,SP500.AAPL,SP500.ABBV,SP500.ABC,SP500.ABMD,SP500.ABT,SP500.ACN,SP500.ADBE,...,SP500.XEL,SP500.XLNX,SP500.XOM,SP500.XRAY,SP500.XRX,SP500.XYL,SP500.YUM,SP500.ZBH,SP500.ZION,SP500.ZTS
alpha_gt_1,0.030283,-0.0014322,0.0173442,0.0206696,-0.016095,0.0250428,-0.011699,0.0135408,-0.00249801,0.0321714,...,0.0369054,-0.00981261,0.0347117,0.00871621,0.0424738,0.0016348,-0.0109862,-0.0159699,-0.00327812,-0.0122796


In [15]:
decay_linear(dp.close, 2)

/Users/fangxia/Library/Mobile Documents/com~apple~CloudDocs/code/hans_radish/paprika/alpha/utils.py:104: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  return df.rolling(p).apply(lambda x: sum(x * w))


Symbol,SP500.A,SP500.AAL,SP500.AAP,SP500.AAPL,SP500.ABBV,SP500.ABC,SP500.ABMD,SP500.ABT,SP500.ACN,SP500.ADBE,...,SP500.XEL,SP500.XLNX,SP500.XOM,SP500.XRAY,SP500.XRX,SP500.XYL,SP500.YUM,SP500.ZBH,SP500.ZION,SP500.ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-04,24.561277,57.146666,35.656668,12.060000,NaN,23.085000,14.150000,23.644544,36.653334,40.219999,...,23.436667,23.796667,73.646668,30.020000,44.523496,NaN,21.138269,77.673332,83.033336,NaN
2007-01-05,24.537434,58.656667,35.546667,12.208095,NaN,23.051667,14.280000,23.942020,37.026667,40.753333,...,23.293334,24.096667,72.893333,30.080000,44.365392,NaN,21.038821,78.803332,83.196666,NaN
2007-01-08,24.356223,58.170001,35.060000,12.170000,NaN,22.985000,14.130000,23.970808,36.856666,40.563333,...,22.980000,24.000001,73.043333,30.136667,44.330259,NaN,20.899832,78.599999,83.076665,NaN
2007-01-09,24.308535,57.920001,35.239999,12.548095,NaN,23.200000,14.060000,24.098754,37.179999,40.176668,...,22.976666,24.063333,72.463333,30.253333,44.778219,NaN,20.892643,78.276667,83.323331,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-05,76.123332,36.829999,161.386673,172.226664,78.916667,84.889997,335.033336,72.703334,156.806671,254.766672,...,52.386668,110.459999,75.076665,42.926668,28.780000,70.459999,94.580002,118.160001,48.949999,87.103335
2019-02-06,76.306666,36.873333,160.046672,174.199997,79.486669,86.059998,344.446666,73.236667,157.266673,254.663340,...,52.410001,112.076668,75.479998,43.223334,29.130000,71.013331,94.490000,118.273333,48.880001,87.440003
2019-02-07,75.976669,36.563331,159.190002,173.140004,79.110001,86.360000,343.723328,73.239998,156.500000,254.146672,...,52.663334,113.756666,75.066668,43.133334,29.280000,71.163333,94.596664,118.216667,49.086666,87.123334
